# Prototype (mostly copy and paste) of LangChain agent code

Please integrate to main script later. Or don't.

### I want to log stuff to make life easier. LangSmith?

LangSmith allows us to "Observe, Evaluate, Develop and Deploy". Is it ClearML tracking for LLMs? Yes. Is it inferior? Of course! Nothing is better than TransparentML!

LangSmith supports searching online or retrieving over local index. This is cool, but I'm gonna ignore Tavily for now.

In [1]:
!pip install -U langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 KB 3.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.131
    Uninstalling langsmith-0.1.131:
      Successfully uninstalled langsmith-0.1.131


### Setup of env, data, embedder, etc.

Available NVIDIA api keys;

1. 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
2. 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

In [ ]:
import getpass
import os

nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

LangSmith API key(s?):

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

In [2]:
!export LANGCHAIN_TRACING_V2=true
!export LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
!export LANGCHAIN_API_KEY="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"
!export LANGCHAIN_PROJECT="cvu-os-NIMRAG"

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="NV-Embed-QA", truncate="END")
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", max_tokens=1024)

# Can choose any model hosted at Nvidia API Catalog (Uncomment the below code to list the availabe models)
# ChatNVIDIA.get_available_models()

In [ ]:
from langchain_community.vectorstores import FAISS

# Load the FAISS vectorestore back.
VECTOR_STORE = './data/nv_embedding'
store = FAISS.load_local(VECTOR_STORE, embedder, allow_dangerous_deserialization=True)
retriever = store.as_retriever()

Now, we can initialize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). For more information about how to think about these components, see LangChain's [conceptual guide](https://python.langchain.com/v0.1/docs/modules/agents/concepts/)

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

tools = [retriever_tool] # list of possible tools for the agent to use. right now only one.

In [ ]:
# placeholder code: we should define our own guiding prompts

from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
# makes the agent and agentexecutor

from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# example questions

agent_executor.invoke({"input": "hi!"})
agent_executor.invoke({"input": "how can langsmith help with testing?"})

### Adding in memory

This agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in previous chat_history. Note: it needs to be called chat_history because of the prompt we are using. If we use a different prompt, we could change the variable name.

In [ ]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})

In [ ]:
# another example of langchain agent with memory
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)

In [ ]:
# automatically keep track of these messages with RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)